In [1]:
import pandas as pd
import numpy as np
import sqlite3
from sklearn.decomposition import TruncatedSVD
import scipy
import threading

In [2]:
svd_predicts = rating_data = movie_data = []

In [3]:
con = sqlite3.connect("db.sqlite3")

movie_data = pd.read_sql_query("SELECT id,title from movies_movie", con)
rating_data = pd.read_sql_query("SELECT * from movies_rating", con)
rating_data.drop('id', axis=1, inplace=True)

movie_data.rename(columns = {'id' : 'movie_id'}, inplace = True)

user_movie_ratings = rating_data.pivot(
    index='user_id',
    columns='movie_id',
    values='rank'
).fillna(0)

matrix = user_movie_ratings.values

user_rating_mean = np.mean(matrix, axis = 1)

matrix_user_mean = matrix - user_rating_mean.reshape(-1,1)

U, sigma, Vt = scipy.sparse.linalg.svds(matrix_user_mean)
sigma = np.diag(sigma)

svd_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_rating_mean.reshape(-1,1)

svd_predicts = pd.DataFrame(svd_user_predicted_ratings, columns = user_movie_ratings.columns)

In [4]:
def user_recommend(user_index,user_id):
    try:
        sorted_user_prediction = svd_predicts.iloc[user_index].sort_values(ascending=False)
        user_data = rating_data[rating_data.user_id == (user_id)]
        user_history = user_data.merge(movie_data, on='movie_id').sort_values(['rank'], ascending=False)
        recommendations = movie_data[~movie_data['movie_id'].isin(user_history['movie_id'])]

        recommendations = recommendations.merge(pd.DataFrame(sorted_user_prediction).reset_index(), on='movie_id')

        recommendations = recommendations.rename(columns = {user_index:'Predictions'}).sort_values('Predictions', ascending=False)
        # print("recommendations= ", recommendations, sep='\n')

        return user_history, recommendations.head(20)
    except:
        return [], []

In [16]:
from tabulate import tabulate

In [24]:
print(svd_predicts)

movie_id        5             11            12            13      \
0         9.999829e-01  5.997190e+00  9.994956e+00  9.999508e+00   
1        -8.590364e-22  1.235987e-18  2.225464e-18  2.209698e-17   
2         6.290040e-05  1.032190e-02  1.852910e-02  1.807332e-03   
3         2.081784e-04  3.416189e-02  6.132486e-02  5.981640e-03   
4         1.395458e-05  2.289933e-03  4.110716e-03  1.000040e+01   
5         1.074525e-04  1.763286e-02  3.165318e-02  3.087458e-03   
6        -1.174113e-04 -1.926709e-02 -3.458683e-02 -3.373607e-03   
7        -5.280005e-18 -4.489749e-16 -8.039308e-16 -6.652161e-15   
8        -6.721604e-04 -1.103009e-01 -1.980039e-01 -1.931334e-02   

movie_id        15            16            18            19      \
0         9.999829e-01  9.999829e-01  1.035034e+00  9.999829e-01   
1        -8.590364e-22 -8.590364e-22  2.550755e-17 -8.590364e-22   
2         6.290040e-05  6.290040e-05 -1.286974e-01  6.290040e-05   
3         2.081784e-04  2.081784e-04 -4.259436e

In [17]:
history, recc = user_recommend(3, 3)
print(tabulate(history, headers='keys', tablefmt='psql', showindex=True))

+----+--------+------------+-----------+-----------------------------------------------+
|    |   rank |   movie_id |   user_id | title                                         |
|----+--------+------------+-----------+-----------------------------------------------|
| 36 |     10 |     496243 |         3 | 기생충                                        |
| 22 |     10 |     299536 |         3 | 어벤져스: 인피니티 워                         |
| 25 |     10 |     284052 |         3 | 닥터 스트레인지                               |
|  1 |     10 |     760517 |         3 | 모럴센스                                      |
| 27 |     10 |     791373 |         3 | 잭 스나이더의 저스티스 리그                   |
| 28 |     10 |     284053 |         3 | 토르: 라그나로크                              |
| 15 |     10 |     429617 |         3 | 스파이더맨: 파 프롬 홈                        |
| 29 |     10 |     363088 |         3 | 앤트맨과 와스프                               |
| 13 |     10 |     102899 |         3 | 앤트맨                                   

In [18]:
print(tabulate(recc, headers='keys', tablefmt='psql', showindex=True))

+------+------------+--------------------------------------+---------------+
|      |   movie_id | title                                |   Predictions |
|------+------------+--------------------------------------+---------------|
|  363 |        671 | 해리 포터와 마법사의 돌              |      10.4711  |
|  364 |        672 | 해리 포터와 비밀의 방                |       9.5471  |
|   61 |        122 | 반지의 제왕: 왕의 귀환               |       9.45397 |
|   59 |        120 | 반지의 제왕: 반지 원정대             |       9.3292  |
| 4127 |      49051 | 호빗: 뜻밖의 여정                    |       8.74843 |
| 5397 |     259316 | 신비한 동물사전                      |       7.80536 |
| 2617 |      12444 | 해리 포터와 죽음의 성물 1            |       7.67902 |
|    7 |         22 | 캐리비안의 해적: 블랙펄의 저주       |       7.627   |
| 5769 |     315635 | 스파이더맨: 홈커밍                   |       7.40082 |
|  274 |        559 | 스파이더맨 3                         |       7.40082 |
| 6862 |     447404 | 명탐정 피카츄                        |       7.33971 |
|  365 |        